<a href="https://colab.research.google.com/github/lorenzoram/aiayn/blob/main/aiayn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction

The paper "Attention Is All You Need" introduces the Transformer model, a breakthrough in deep learning architecture for tasks involving sequences, such as translation and text generation. The key contributions are:

- Attention Mechanisms: The model relies solely on attention mechanisms, dispensing with the need for recurrence (RNNs) or convolutions (CNNs).
- Parallelization: The design allows for significant parallelization, making the model more efficient to train.
- Performance: The Transformer achieves state-of-the-art results in machine translation tasks, outperforming previous models in both speed and accuracy.

# Understanding




Main Concepts
- Self-Attention: Each word in a sequence is compared to every other word, allowing the model to focus on different parts of the input sequence when generating each word of the output.
- Multi-Head Attention: Multiple attention mechanisms run in parallel, enabling the model to capture different aspects of the input sequence simultaneously.

Methods
- Encoder-Decoder Structure:
  - Encoder: Consists of layers that apply self-attention and feed-forward neural networks to transform the input sequence into a set of continuous representations.
  - Decoder: Uses these representations to generate the output sequence, applying self-attention to the output and attention to the encoder's output.

- Positional Encoding: Adds information about the order of words in the sequence, addressing the lack of sequential information in the model's architecture.

Results

- BLEU Scores: The Transformer outperforms previous models in BLEU scores (a metric for evaluating the quality of text which has been machine-translated from one language to another), particularly in English-to-German and English-to-French translation tasks.
- Efficiency: Due to its parallelizable architecture, the Transformer trains faster and requires less computational resources compared to RNNs and CNNs.

By leveraging attention mechanisms, the Transformer model introduces a more efficient and powerful way to handle sequence transduction tasks, setting a new standard in natural language processing.

# Reproduction

# Extension

# Conclusion